In [2]:
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, hamming_loss, make_scorer # type: ignore


## Read fingerprint file and experimental file

In [3]:
receptors = pd.read_csv('/Users/xiaomuou620/Desktop/PRIVATE_DATA/selected_file.csv') # selected only existing binding receptors left
fingerprints = pd.read_csv('/Users/xiaomuou620/Library/CloudStorage/OneDrive-UniversityofCopenhagen/courses/Project1/Fingerprint/MACCS_fingerprints.csv')
print(receptors.head())



  receptor gprotein       drug        Emax    Emax_SE     TCoeff  TCoeff_SE  \
0    5HT1A      GoB  25C-NBOMe   13.047601   1.862662   7.863330   0.215495   
1    5HT1D      Gi2  25C-NBOMe   61.560000  16.200000   7.308909   0.275469   
2    5HT2A    BArr2  25C-NBOMe  126.520000   2.660000  10.852095   0.045687   
3    5HT2A      G11  25C-NBOMe   97.880000   1.930000  11.570638   0.051133   
4    5HT2A      G15  25C-NBOMe   89.690000   2.150000  11.482661   0.057973   

     logEmEC  logEmEc_Lower  logEmEc_Upper    pEC50  pEC50_SE  N  
0   7.866311       7.488600       8.235079  6.75078  0.310814  3  
1   7.319299       6.826673       7.780756  5.53000  0.360000  3  
2  10.852159      10.772931      10.931195  8.75000  0.070000  3  
3  11.570694      11.482045      11.659174  9.58000  0.080000  3  
4  11.482744      11.382207      11.583032  9.53000  0.090000  3  


### Experimental data processing

In [4]:
# collect receptors for each drug
drug_receptor_mapping = receptors.groupby('drug')['receptor'].apply(list).reset_index()
print(drug_receptor_mapping.head())

         drug                                           receptor
0   25C-NBOMe  [5HT1A, 5HT1D, 5HT2A, 5HT2A, 5HT2A, 5HT2A, 5HT...
1    25I-NBMD  [5HT1A, 5HT1A, 5HT1A, 5HT1B, 5HT1B, 5HT1B, 5HT...
2   25I-NBOMe  [5HT1A, 5HT1A, 5HT1D, 5HT1D, 5HT1D, 5HT1D, 5HT...
3  25T7-NBOMe  [5HT1A, 5HT1A, 5HT1A, 5HT1D, 5HT1D, 5HT1D, 5HT...
4  4-AcO-MALT  [5HT1A, 5HT1A, 5HT1A, 5HT1A, 5HT1A, 5HT1B, 5HT...


In [5]:
# binarize the receptors
mlb = MultiLabelBinarizer()
receptor_labels = mlb.fit_transform(drug_receptor_mapping['receptor'])

In [6]:
drug_receptor_label = pd.concat([drug_receptor_mapping, pd.DataFrame(receptor_labels, columns=mlb.classes_)], axis=1)
print(drug_receptor_label.head())

         drug                                           receptor  5HT1A  \
0   25C-NBOMe  [5HT1A, 5HT1D, 5HT2A, 5HT2A, 5HT2A, 5HT2A, 5HT...      1   
1    25I-NBMD  [5HT1A, 5HT1A, 5HT1A, 5HT1B, 5HT1B, 5HT1B, 5HT...      1   
2   25I-NBOMe  [5HT1A, 5HT1A, 5HT1D, 5HT1D, 5HT1D, 5HT1D, 5HT...      1   
3  25T7-NBOMe  [5HT1A, 5HT1A, 5HT1A, 5HT1D, 5HT1D, 5HT1D, 5HT...      1   
4  4-AcO-MALT  [5HT1A, 5HT1A, 5HT1A, 5HT1A, 5HT1A, 5HT1B, 5HT...      1   

   5HT1B  5HT1D  5HT1E  5HT1F  5HT2A  5HT2B  5HT2C  ...  Alpha2A  Alpha2B  \
0      0      1      0      0      1      1      1  ...        0        0   
1      1      1      1      1      1      1      1  ...        0        0   
2      0      1      1      1      1      1      1  ...        0        0   
3      0      1      0      1      1      1      1  ...        0        0   
4      1      1      1      1      1      1      1  ...        0        0   

   Alpha2C  Beta2AR  Beta3AR  DRD1  DRD2  DRD3  DRD4  DRD5  
0        0        0      

In [7]:
drug_label = drug_receptor_label.drop(columns=['receptor'])
print(drug_label.head())
drug_label.to_csv('/Users/xiaomuou620/Desktop/PRIVATE_DATA/selected_file_multilabel.csv', index=False)

# df_indexed = drug_label.set_index('drug')
drug_dict = drug_label.to_dict()
print(drug_dict)

         drug  5HT1A  5HT1B  5HT1D  5HT1E  5HT1F  5HT2A  5HT2B  5HT2C  5HT5A  \
0   25C-NBOMe      1      0      1      0      0      1      1      1      0   
1    25I-NBMD      1      1      1      1      1      1      1      1      0   
2   25I-NBOMe      1      0      1      1      1      1      1      1      1   
3  25T7-NBOMe      1      0      1      0      1      1      1      1      1   
4  4-AcO-MALT      1      1      1      1      1      1      1      1      1   

   ...  Alpha2A  Alpha2B  Alpha2C  Beta2AR  Beta3AR  DRD1  DRD2  DRD3  DRD4  \
0  ...        0        0        0        0        0     0     1     0     0   
1  ...        0        0        0        0        0     0     0     0     0   
2  ...        0        0        0        0        0     0     0     1     0   
3  ...        0        0        0        0        0     0     0     1     0   
4  ...        0        0        0        0        0     0     1     1     0   

   DRD5  
0     0  
1     0  
2     0  
3   

### Fingerprint data processing

In [8]:
# Extract the names (first column)
drug_names_fingerprints = fingerprints['Name']
fingerprints_detail = fingerprints.drop('Name', axis=1)  # Drop the name column by index

# Create a dictionary where names are keys and fingerprints are lists of values
# csv_dict = {name: fingerprint.tolist() for name, fingerprint in zip(drug_names_fingerprints, fingerprints_detail.values)}


### Merge feature and labels

In [10]:
data_merged = pd.merge(drug_label, fingerprints, left_on='drug', right_on='Name').drop(columns=['Name'])
# data_merged.to_csv('/Users/xiaomuou620/Desktop/PRIVATE_DATA/data_merged.csv', index=False)


## Training

In [11]:
X = data_merged.drop(columns=['drug'] + list(mlb.classes_))
y = data_merged[list(mlb.classes_)]

print(X.shape, y.shape)


(40, 167) (40, 24)


In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

## Optuna

In [13]:
import optuna
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, hamming_loss, f1_score, jaccard_score
from sklearn.model_selection import train_test_split


# define the objective function
def objective(trial):
    # define the hyperparameters to optimize
    n_estimators = trial.suggest_int("n_estimators", 10, 200)
    max_depth = trial.suggest_int("max_depth", 5, 30)
    min_samples_split = trial.suggest_int("min_samples_split", 2, 10)
    min_samples_leaf = trial.suggest_int("min_samples_leaf", 1, 4)
    bootstrap = trial.suggest_categorical("bootstrap", [True, False])
    
    # create a random forest classifier with the specified hyperparameters
    rf_classifier = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        bootstrap=bootstrap,
        random_state=42
    )
    
    # train the model
    rf_classifier.fit(X_train, y_train)
    
    # make predictions on the test set
    y_pred = rf_classifier.predict(X_test)
    
    # evaluate the model using the Hamming loss metric
    ''' ? should i minimize the hamming loss'''
    hamming_loss_score = hamming_loss(y_test, y_pred) 
    
    
    # return the Hamming loss score
    return hamming_loss_score

# use Optuna to optimize the hyperparameters
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=100) 

# output the best hyperparameters   
print("Best Hyperparameters:", study.best_params)

# train a random forest classifier with the best hyperparameters
best_params = study.best_params
best_rf_classifier = RandomForestClassifier(**best_params, random_state=42)
best_rf_classifier.fit(X_train, y_train)

# make predictions on the test set
y_test_pred = best_rf_classifier.predict(X_test)

print("\n--- Final Evaluation on Test Set with Best Hyperparameters ---")
print(f"Test Accuracy: {accuracy_score(y_test, y_test_pred):.2f}")
print(f"Test Hamming Loss: {hamming_loss(y_test, y_test_pred):.2f}")
print(f"Test F1 Score: {f1_score(y_test, y_test_pred, average='micro'):.2f}")
print(f"Test Jaccard Index (samples average): {jaccard_score(y_test, y_test_pred, average='samples'):.2f}")


/Users/xiaomuou620/opt/miniconda3/envs/Project1/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2024-11-20 14:57:32,637] A new study created in memory with name: no-name-f4d1967c-d394-48cf-9040-ab7b9f3b6d8c
[I 2024-11-20 14:57:32,752] Trial 0 finished with value: 0.13194444444444445 and parameters: {'n_estimators': 117, 'max_depth': 5, 'min_samples_split': 7, 'min_samples_leaf': 1, 'bootstrap': False}. Best is trial 0 with value: 0.13194444444444445.
[I 2024-11-20 14:57:32,789] Trial 1 finished with value: 0.125 and parameters: {'n_estimators': 39, 'max_depth': 13, 'min_samples_split': 3, 'min_samples_leaf': 2, 'bootstrap': False}. Best is trial 1 with value: 0.125.
[I 2024-11-20 14:57:32,917] Trial 2 finished with value: 0.125 and parameters: {'n_estimators': 131, 'max_depth': 16, 'min_samples_split'

Best Hyperparameters: {'n_estimators': 120, 'max_depth': 24, 'min_samples_split': 2, 'min_samples_leaf': 1, 'bootstrap': False}

--- Final Evaluation on Test Set with Best Hyperparameters ---
Test Accuracy: 0.17
Test Hamming Loss: 0.11
Test F1 Score: 0.91
Test Jaccard Index (samples average): 0.82


In [14]:
from sklearn.metrics import classification_report
report = classification_report(y_test, y_test_pred, target_names=y.columns)
print(report)


              precision    recall  f1-score   support

       5HT1A       1.00      1.00      1.00        12
       5HT1B       1.00      1.00      1.00        12
       5HT1D       1.00      1.00      1.00        12
       5HT1E       1.00      0.92      0.96        12
       5HT1F       1.00      1.00      1.00        12
       5HT2A       1.00      1.00      1.00        12
       5HT2B       1.00      1.00      1.00        12
       5HT2C       1.00      1.00      1.00        12
       5HT5A       0.92      1.00      0.96        11
        5HT6       1.00      0.88      0.93         8
        5HT7       0.60      0.75      0.67         4
     Alpha1A       0.88      0.88      0.88         8
     Alpha1B       0.50      0.75      0.60         4
     Alpha1D       0.80      0.80      0.80         5
     Alpha2A       0.75      1.00      0.86         3
     Alpha2B       1.00      0.67      0.80         6
     Alpha2C       0.50      1.00      0.67         2
     Beta2AR       0.00    

/Users/xiaomuou620/opt/miniconda3/envs/Project1/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/xiaomuou620/opt/miniconda3/envs/Project1/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## GridSearchCV

In [15]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report


# define the random forest model
rf_model = RandomForestClassifier(random_state=42)

# define the hyperparameters to optimize
param_grid = {
    'n_estimators': [10, 100, 300],  
    'max_depth': [None, 10, 30, 50],  
    'min_samples_split': [2, 10, 20],  
    'min_samples_leaf': [1, 4, 6, 8],      
    'bootstrap': [True, False]                
}

# define the grid search with 5-fold cross-validation
grid_search = GridSearchCV(estimator=rf_model, param_grid=param_grid, cv=5, scoring='f1_samples', n_jobs=-1)

# perform the grid search
grid_search.fit(X_train, y_train)

# output the best hyperparameters
print("Best Parameters:", grid_search.best_params_)
print("Best Cross-Validation Score:", grid_search.best_score_)


Best Parameters: {'bootstrap': False, 'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 10, 'n_estimators': 10}
Best Cross-Validation Score: 0.8840541918733826


In [16]:
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)
hamming_loss_score_grid = hamming_loss(y_test, y_pred)
print("Test Score with Best Model:", hamming_loss_score_grid)

report = classification_report(y_test, y_pred, target_names=y.columns)
print(report)

Test Score with Best Model: 0.13194444444444445
              precision    recall  f1-score   support

       5HT1A       1.00      1.00      1.00        12
       5HT1B       1.00      1.00      1.00        12
       5HT1D       1.00      1.00      1.00        12
       5HT1E       1.00      1.00      1.00        12
       5HT1F       1.00      1.00      1.00        12
       5HT2A       1.00      1.00      1.00        12
       5HT2B       1.00      1.00      1.00        12
       5HT2C       1.00      1.00      1.00        12
       5HT5A       0.92      1.00      0.96        11
        5HT6       1.00      0.88      0.93         8
        5HT7       0.50      0.50      0.50         4
     Alpha1A       1.00      0.88      0.93         8
     Alpha1B       0.00      0.00      0.00         4
     Alpha1D       1.00      0.80      0.89         5
     Alpha2A       0.75      1.00      0.86         3
     Alpha2B       1.00      0.67      0.80         6
     Alpha2C       0.50      1.00

/Users/xiaomuou620/opt/miniconda3/envs/Project1/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/xiaomuou620/opt/miniconda3/envs/Project1/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [32]:
# 

[[0.01545635 0.98454365]
 [0.01545635 0.98454365]
 [0.01833333 0.98166667]
 [0.03382143 0.96617857]
 [0.03296429 0.96703571]
 [0.01545635 0.98454365]
 [0.0141746  0.9858254 ]
 [0.01545635 0.98454365]
 [0.05557143 0.94442857]
 [0.07083333 0.92916667]
 [0.03382143 0.96617857]
 [0.00333333 0.99666667]]
